In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install transformers

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
file_path = "/content/drive/MyDrive/my-datasets/yektanet_train.csv"

data = pd.read_csv(file_path)

# data Preprocessing
def preprocess_data(df):
   # Get unique categories
    categories = df['category'].unique()

    # Create new dataframe with all columns except 'category'
    new_df = df.copy()

    # Create binary columns for each category
    for category in categories:
        # Create new column with category name and fill with 0s and 1s
        new_df[category] = (df['category'] == category).astype(int)

    # Drop the original category column
    new_df = new_df.drop('category', axis=1)

    return new_df

proccessed_data = preprocess_data(data)

# get all categories
non_category_columns = ['description', 'text_content', 'title']
category_columns = [col for col in proccessed_data.columns if col not in non_category_columns]

proccessed_data["content"] = proccessed_data['title'] + "-" + proccessed_data['text_content'] + "-" + proccessed_data['description']


# drop useless columns
cols_to_drop = ["title", "text_content","description","url", "domain", "id","h1","h2"]
existing_cols = [col for col in cols_to_drop if col in proccessed_data.columns]
proccessed_data.drop(labels=existing_cols, axis=1, inplace=True)

proccessed_data.head()


,کتاب و ادبیات,تجارت و اقتصاد,سلامت,تکنولوژی و کامپبوتر,ورزش,غذا و نوشیدنی,حقوق و دولت و سیاست,مسکن,خودرو,مد و زیبایی,...,سفر و گردشگری,هنر و سرگرمی,خانه و باغبانی,موسیقی,تحصیلات,علم و دانش,خانواده,اشتغال,حیوانات خانگی,content
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,معنی از شوبنده ها | جدول یاب- معنی از شوبنده ...
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,عکس بیت‌کوین کش برای پروفایل-عکس بیت‌کوین کش ب...
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,دانلود تحریم‌گذر Geph برای اندروید-دانلود تحری...
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...


In [5]:
features = proccessed_data['content']

target_list = proccessed_data.drop('content',axis=1)

x_train, x_test, y_train, y_test = train_test_split(
    features, target_list, test_size=0.2, random_state=42, stratify=target_list
)

print("Training set shape:", x_train.shape)
print("Test set shape:", x_test.shape)

# print("Training set shape:", y_train.shape)
# print("Test set shape:", y_test.shape)

Training set shape: (3831,)
Test set shape: (958,)


In [6]:
# implement hypo parameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

In [7]:
from transformers import BertTokenizer,BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
# [CLS] , [PAD] , [SEP] , [MASK]

example_text = "My name is hossein"

encodings = tokenizer.encode_plus(
    example_text,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

encodings

{'input_ids': tensor([[  101,  2026,  2171,  2003,  7570, 11393,  2378,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.title = df['content'].astype(str)
        # get all columns except 'content' as they are the target categories
        self.category_columns = [col for col in df.columns if col != 'content']
        self.targets = df[self.category_columns].values

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [10]:
train_size = 0.8

print(x_train)

train_df = pd.concat([x_train, y_train], axis=1)
train_df = train_df.sample(frac=train_size, random_state=200).reset_index(drop=True)

val_df = pd.concat([x_test, y_test], axis=1)
val_df = val_df.reset_index(drop=True)  # Changed this line, removed the drop operation

3380    Mamad ahmadi-Mamad ahmadi Skip to content شبکه...
1457    خواص عاقرقرحا ؛ نحوه استفاده از عاقرقرحا در طب...
1473    روزنامه کيهان:                                ...
3316    آغاز ثبت نام طرح‌های فروش محصولات ایران‌خودرو-...
148     تور خارجی | ترکیه 🕌 ، امارات ، تایلند ، مالزی ...
                              ...                        
2861    روش درست برخورد با بیماری همسر چیست؟-روش درست ...
3314    یاسمین مقبلی، زن ایرانی که به ماه می‌رود - بها...
4104    بذر کاکتوس های ستونی میکس بسته ۱۰ عددی - میهن ...
4519    خرید و قیمت کتاب قصه ها و غصه های زال زر اثر ع...
3832    ماهنامه شمارۀ 50 و 51 پیش به سوی کشاورزی اقتصا...
Name: content, Length: 3831, dtype: object


In [11]:
train_dataset = CustomDataset(train_df,tokenizer,MAX_LEN)
valid_dataset = CustomDataset(val_df,tokenizer,MAX_LEN)

In [12]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    shuffle=False,
    batch_size=VALID_BATCH_SIZE,
    num_workers=0
)

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [14]:
def load_ckp(checkpoint_fpath,model,optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'],valid_loss_min.item()


def save_ckp(state,is_best,checkpoint_path,best_model_path):
    f_path = checkpoint_path
    torch.save(state,f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path,best_fpath)

In [15]:
class BERTClass(nn.Module):
    def __init__(self, num_categories):  # Add num_categories parameter
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768, num_categories)  # Change 6 to num_categories

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert_model(input_ids, attention_mask, token_type_ids)
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

# Get the number of categories
num_categories = len([col for col in train_df.columns if col != 'content'])
# Initialize the model with the correct number of categories
model = BERTClass(num_categories)
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [16]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs,targets)

optimizer = torch.optim.Adam(params=model.parameters(),lr=LEARNING_RATE)

In [17]:
def train_model(
    n_epochs,
    training_loader,
    validation_loader,
    model,
    optimizer,
    loss_fn,
    device,
    checkpoint_path,
    best_model_path
):
    valid_loss_min = float('inf')

    for epoch in range(1, n_epochs + 1):
        train_loss = 0.0
        valid_loss = 0.0

        model.train()
        for batch_idx, batch in enumerate(training_loader):
            input_ids = batch['input_ids'].to(device, dtype=torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.float)

            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask, token_type_ids)

            loss = loss_fn(outputs, targets)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss = train_loss / len(training_loader)

        model.eval()
        with torch.no_grad():
            for batch_idx, batch in enumerate(validation_loader):
                input_ids = batch['input_ids'].to(device, dtype=torch.long)
                attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
                token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
                targets = batch['targets'].to(device, dtype=torch.float)

                outputs = model(input_ids, attention_mask, token_type_ids)

                loss = loss_fn(outputs, targets)

                valid_loss += loss.item()

        valid_loss = valid_loss / len(validation_loader)

        print(f'Epoch: {epoch}/{n_epochs}')
        print(f'Training Loss: {train_loss:.4f}')
        print(f'Validation Loss: {valid_loss:.4f}')

        checkpoint = {
            'epoch': epoch,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        if valid_loss < valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss
        else:
            save_ckp(checkpoint, False, checkpoint_path, best_model_path)

    return model

In [18]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

model = train_model(
    n_epochs=EPOCHS,
    training_loader=train_data_loader,
    validation_loader=val_data_loader,
    model=model,
    optimizer=optimizer,
    loss_fn=criterion,
    device=device,
    checkpoint_path='checkpoint.pt',
    best_model_path='best_model.pt'
)

Epoch: 1/2
Training Loss: 0.3994
Validation Loss: 0.2702
Validation loss decreased (inf --> 0.270242). Saving model...
Epoch: 2/2
Training Loss: 0.2396
Validation Loss: 0.2047
Validation loss decreased (0.270242 --> 0.204683). Saving model...


In [53]:
example = x_test.iloc[1]
print(example)

encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

model.eval()
with torch.no_grad():
     input_ids = encodings['input_ids'].to(device, dtype=torch.long)
     attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
     token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
     output = model(input_ids,attention_mask,token_type_ids)
     final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
     print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

دکتر منصور خالقی متخصص چشم پزشک در بوشهر-دکتر منصور خالقی متخصص چشم پزشک در بوشهر دکتریاب نوبت دهی پزشکان عضویت پزشکان ورود پزشکان مشاوره تلفنی پرسش از پزشکان ورود کاربران دکتریاب مطب پزشکان بوشهردکتر چشم پزشک بوشهردکتر منصور خالقی دکتر منصور خالقی متخصص چشم پزشک در بوشهر (21 رای) پیشنهاد میـکنم پیشنهاد نـمی کنم follow اشتراک آدرس و تلفن دکتر منصور خالقی 077-33539898 بوشهر خیابان یادگار امام ، ساختمان پزشکان نور آیا اطلاعات اشتباه یا قدیمی است! اینجا کلیک کنید در صورت مطلع بودن از آدرس و شماره جدید آن را در کادر زیر وارد نمایید و در غیر این صورت تنها ارسال را بفشارید. لطفا از ارسال اطلاعات کذب جدا خودداری کنید ارسال ⚠️اطلاعات درج شده در این صفحه صرفا بازنشر شده از سایت نظام پزشکی و یا جمع آوری شده از اینترنت است و دکتریاب تایید کننده صحت آن نیست و لذا بهتر است کاربر قبل از مراجعه تحقیقات بیشتری جهت اطمینان انجام دهد. قوانین و تذکرات لیست متخصص چشم پزشک در خیابان یادگار امام پزشکان پیشنهادی دکتر محمد احمدحسینی متخصص چشم پزشک بوشهر، میدان امام دکتر محمدحسین صادقی متخصص چشم پزشک بوشهر، می

<ipython-input-53-752bd0f8389f>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
